# Energy determination of parameters

In [2]:
import numpy as np
import veloxchem as vlx

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 4.


## Order expansion of parameters

The wave function parameters are expanded in orders of the external field

$$
\kappa_{ai}(F) =
\kappa_{ai}^{(1)} +
\kappa_{ai}^{(2)} + \ldots
$$

such that $\kappa_{ai}= 0$ in the limit of $F = 0$.

Conversely, 

$$
\kappa_{ai}^{(1)} =
\left.
\frac{
d \kappa_{ai}
}{
d F_{\alpha}
}
\right|_{F=0} F_{\alpha} 
$$

## Linear response equation

The first-order response can be determined from the field-derivative of the variational condition

\begin{align*}
0 & =
\Big[
\frac{
d
}{
d F_{\alpha}
} 
\frac{\partial E(\kappa, F)}{\partial \kappa_{bj}^*}
\Big]_{F=0} 
\\ & = 
\Big[
\frac{\partial^2 E(\kappa, F)}{\partial \kappa_{bj}^* \partial F_\alpha}
+
\sum_{ai} 
\Big(
\frac{
\partial^2 E(\kappa, F)
}{
\partial \kappa_{bj}^* \partial \kappa_{ai}
}
\frac{\partial \kappa_{ai}}{\partial F_\alpha} 
+
\frac{
\partial^2 E(\kappa, F)
}{
\partial \kappa_{bj}^* \partial \kappa_{ai}^*
}
\frac{\partial \kappa_{ai}^*}{\partial F_\alpha} 
\Big)
\Big]_{F=0} 
\end{align*}

We here view the $ai$-pair of unoccupied/occupied orbital indices as a compound index $n$ (and similarly for $bj$ and $m$), inviting us to introduce a compact matrix notation to arrive at the linear response equation.

$$
\mathbf{E}^{[2]} \mathbf{N} = - \mathbf{B}^{[1]}
$$

In order, we have introduced:

1. The Hermitian electronic Hessian
$$
\mathbf{E}^{[2]} = 
\begin{pmatrix}
\mathbf{A} & \mathbf{B} \\
\mathbf{B}^* & \mathbf{A}^* \\
\end{pmatrix}
$$
with blocks
$$
A_{mn} =
\left.
\frac{
\partial^2 E(\kappa, F)
}{
\partial \kappa_{m}^* \partial \kappa_{n}
}
\right|_{F = 0} ; \qquad
B_{mn} =
\left.
\frac{
\partial^2 E(\kappa, F)
}{
\partial \kappa_{m}^* \partial \kappa_{n}^*
}
\right|_{F = 0} 
$$

2. The (unknown) response vector
$$
\mathbf{N} =
\begin{pmatrix}
\mathbf{Z} \\
\mathbf{Z}^*
\end{pmatrix}
$$
with 
$$
Z_n = 
\left.
\frac{\partial \kappa_n}{\partial F_\alpha}
\right|_{F=0}
$$

3. And the property gradient
$$
\mathbf{B}^{[1]} =
\begin{pmatrix}
\mathbf{g} \\
\mathbf{g}^* \\
\end{pmatrix}
$$
with
$$
g_m = 
\left.
\frac{\partial^2 E(\kappa, F)}{\partial \kappa_{m}^* \partial F_\alpha}
\right|_{F=0}
$$

## Baker–Campbell–Hausdorff expansion

We have

\begin{align*}
E(\kappa,F) & =
\langle 0 | e^{i\hat{\kappa}} \hat{H} e^{-i\hat{\kappa}} | 0 \rangle
\\ & =
\langle 0 | \hat{H} | 0 \rangle +
i \langle 0 | [\hat{\kappa}, \hat{H}] | 0 \rangle -
\frac{1}{2}
\langle 0 | [\hat{\kappa}, [\hat{\kappa}, \hat{H}]] | 0 \rangle +
\cdots
\end{align*}

and let us adopt the electric-dipole approximation

$$
\hat{H} = \hat{H}_0 - \hat{\mu}_\alpha F_\alpha
$$

It is then straightforward to evaluate that electronic Hessian and the property gradient. We get

\begin{align*}
A_{mn} & = -
\langle 0 | [\hat{a}^\dagger_j \hat{a}_b, 
[\hat{a}^\dagger_a \hat{a}_i, \hat{H}_0]] | 0 \rangle
\\ & =
\langle 0_j^b | \hat{H}_0 | 0_i^a \rangle -
\langle 0 | \hat{H}_0 | 0 \rangle \delta_{ab} \delta_{ij}
\\
B_{mn} & = -
\langle 0 | [\hat{a}^\dagger_j \hat{a}_b, 
[\hat{a}^\dagger_i \hat{a}_a, \hat{H}_0]] | 0 \rangle
\\ & =
- \langle 0_{ij}^{ab} | \hat{H}_0 | 0 \rangle 
( 1 - \delta_{ab} ) ( 1 - \delta_{ij} )
\end{align*}

and

\begin{align*}
g_m & = i
\langle 0 | [\hat{a}^\dagger_j \hat{a}_b, -\hat{\mu}_\alpha ] | 0 \rangle 
\\ & =
- i \langle 0_j^b | \hat{\mu}_\alpha | 0 \rangle
\\ & =
-i \langle \psi_j | \hat{\mu}_\alpha | \psi_b \rangle 
\end{align*}

## Spin-adapted excitation operators

For closed-shell reference states in VeloxChem, we are using [normalized spin-adapted excitation operators](spin_adapted_excitation_operators) that will result in 

\begin{align*}
A_{mn}  &= (\varepsilon_a - \varepsilon_i) \delta_{ab} \delta_{ij} + 2 (jb|ia) - (ji|ba)
\\
B_{mn}  &= (jb|ai) - (ja|bi)
\end{align*}

and 

\begin{align*}
g_m & = -i \sqrt{2} ( \phi_j | \hat{\mu}_\alpha | \phi_b ) 
\end{align*}

and, in this case, the block dimension equals 

$$
n_\mathrm{dim} = n_\mathrm{occ} \times n_\mathrm{virt}
$$

## Illustration

### Setup

In [3]:
h2o_xyz = """3

O    0.000000000000        0.000000000000        0.000000000000
H    0.000000000000        0.740848095288        0.582094932012
H    0.000000000000       -0.740848095288        0.582094932012
"""

In [47]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()

molecule = vlx.Molecule.read_xyz_string(h2o_xyz)
basis = vlx.MolecularBasis.read(molecule, "6-31g", ostream=None)

scf_results = scf_drv.compute(molecule, basis)

In [62]:
epsilon = scf_results["E_alpha"]  # orbital energies

In [63]:
erimo_drv = vlx.MOIntegralsDriver()  # ERI blocks in the MO basis

ovov = erimo_drv.compute_in_memory(molecule, basis, scf_drv.mol_orbs, "chem_ovov")
oovv = erimo_drv.compute_in_memory(molecule, basis, scf_drv.mol_orbs, "chem_oovv")

print("(ov|ov):", ovov.shape)
print("(oo|vv):", oovv.shape)

(ov|ov): (5, 8, 5, 8)
(oo|vv): (5, 5, 8, 8)


In [108]:
nocc = molecule.number_of_alpha_electrons()
norb = basis.get_dimension_of_basis(molecule)
nvirt = norb - nocc
ndim = nocc * nvirt

print("Number of occupied orbitals:", nocc)
print("Number of virtual orbitals :", nvirt)
print("Block dimension            :", ndim)

Number of occupied orbitals: 5
Number of virtual orbitals : 8
Block dimension            : 40


### Electronic Hessian

We determine the electronic Hessian

$$
\mathbf{E}^{[2]} = 
\begin{pmatrix}
\mathbf{A} & \mathbf{B} \\
\mathbf{B}^* & \mathbf{A}^* \\
\end{pmatrix}
$$

In [104]:
def delta(i, j):

    if i == j:
        delta = 1
    else:
        delta = 0
        
    return delta

In [109]:
A = np.zeros((ndim, ndim))
B = np.zeros((ndim, ndim))

m = 0
for j in range(nocc):
    for b in range(nvirt):
        n = 0
        for i in range(nocc):
            for a in range(nvirt):
                A[m, n] = (
                    (epsilon[a + nocc] - epsilon[i]) * delta(a, b) * delta(i, j)
                    + 2 * ovov[j, b, i, a]
                    - oovv[j, i, b, a]
                )
                #B[m,n] = ...
                n += 1
        m += 1

We ensure that our result equals the electronic Hessian obtained with the `get_e2()` method of the `LinearResponseEigenSolver` class.

In [106]:
lres_drv = vlx.LinearResponseEigenSolver()
lres_drv.ostream.mute()

E2_ref = lres_drv.get_e2(molecule, basis, scf_results)

In [107]:
np.testing.assert_allclose(A, E2_ref[:ndim, :ndim], atol=1.0e-6)

### Property gradient

We determine the property gradient

$$
\mathbf{B}^{[1]} =
\begin{pmatrix}
\mathbf{g} \\
\mathbf{g}^* \\
\end{pmatrix}
$$

for the operator $\hat{\mu}_z$.

In [49]:
dipole_drv = vlx.ElectricDipoleIntegralsDriver()
dipole_mats = dipole_drv.compute(molecule, basis)

mu_z_ao = -1.0 * dipole_mats.z_to_numpy()

C = scf_results["C_alpha"]

mu_z = np.einsum("ap, ab, bq -> pq", C, mu_z_ao, C)

In [50]:
B1 = np.zeros(2 * ndim, dtype=complex)

g = []
for j in range(nocc):
    for b in range(nocc, norb):
        g.append(-1j * np.sqrt(2) * mu_z[j, b])
g = np.array(g)

B1[:ndim] = g
B1[ndim:] = g.conj()

We ensure that our result equals the reference property gradient obtained with the `get_prop_grad()` method of the `LinearResponseSolver` class.

In [95]:
lres_drv = vlx.LinearResponseSolver()

B1_ref = lrs_drv.get_prop_grad(
    "electric_dipole", "z", molecule, basis, scf_drv.scf_tensors
)[0]

In [96]:
np.testing.assert_allclose(B1.imag, B1_ref, atol=1.0e-12)

### Response vector

We determine the response vector

$$
\mathbf{N} = - \Big[ \mathbf{E}^{[2]}  \Big]^{-1}\mathbf{B}^{[1]}
$$

In [10]:
Z = (
    -0.5
    * (
        lrf_results["solutions"][("z", 0.0)].get_full_vector(0)
        + lrf_results["solutions"][("z", 0.0)].get_full_vector(1)
    )
    * np.sqrt(2)
)

Y = (
    -0.5
    * (
        lrf_results["solutions"][("z", 0.0)].get_full_vector(0)
        - lrf_results["solutions"][("z", 0.0)].get_full_vector(1)
    )
    * np.sqrt(2)
)

In [11]:
Z[:10]

array([-8.48351857e-04,  1.61661005e-17, -6.03746199e-17,  2.15254468e-19,
       -4.18679500e-03, -6.98494415e-04, -2.28651384e-17, -1.21381762e-04,
        4.06014225e-02, -4.68699343e-16])

In [12]:
Y[:10]

array([ 8.48351857e-04, -1.61661005e-17,  6.03746199e-17, -2.15254468e-19,
        4.18679500e-03,  6.98494415e-04,  2.28651384e-17,  1.21381762e-04,
       -4.06014225e-02,  4.68699343e-16])